In [1]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Read data into a dataframe and 
# df = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = df.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]
# densityPlot = sns.kdeplot(counts, bw_adjust=3)
# yLabels = densityPlot.get_yticks()
# densityPlot.set_yticklabels('{:,.0%}'.format(y) for y in yLabels)
# plt.xlabel("Number of reviews")
# plt.ylabel("Percentage of users")
# plt.savefig(fname="densityPlot")
# plt.show()

In [2]:
# import pandas as pd

# # Read ratings into a dataframe and narrow down reviewers to desired review count
# ratings = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = ratings.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]

# # Create and apply mask that removes all other reviewers.
# mask = ratings['reviewerID'].isin(counts.index)
# ratings = ratings[mask]

# # Read metadata into dataframe and define asins that are used by making a set
# metadata = pd.read_csv("metadata.csv")
# used_asins = set(ratings["asin"])

# # Create and apply mask that removes unreferenced items
# mask = metadata["asin"].isin(used_asins)
# metadata = metadata[mask]
# metadata.to_pickle("metadata.pickle")
# ratings.to_pickle("ratings.pickle")


In [3]:
# import pandas as pd
# import numpy as np
# from scipy.sparse import csr_matrix, save_npz
# ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
# metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
# ratings = ratings[["reviewerID", "asin", "overall"]]

# # Series of each unique value of reviewerID and asin 
# reviewerIDs = ratings['reviewerID'].unique()
# asins = ratings["asin"].unique()
# # Dictionary that maps reviewers to an index
# reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}

# # Dictionary that maps asins to an index
# asinMap = {asin: i for i, asin in enumerate(asins)}

# # Initialize the utility matrix with zeros
# utilityMatrixArray = np.zeros((len(reviewerIDs), len(asins)), "int8")

# # Fill in the utility matrix with data from the records array
# for row in ratings.itertuples():
#     reviewerIndex = reviewerMap[row.reviewerID]
#     asinIndex = asinMap[row.asin]
#     utilityMatrixArray[reviewerIndex, asinIndex] = row.overall
# utilityMatrix = csr_matrix(utilityMatrixArray)
# save_npz("utilityMatrix.npz", utilityMatrix)

In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, csr_matrix, hstack
from sklearn.metrics.pairwise import cosine_similarity


ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
utilityMatrix = load_npz("utilityMatrix.npz")

# Series of each unique value of reviewerID and asin 
reviewerIDs = ratings['reviewerID'].unique()
asins = ratings["asin"].unique()

# Dictionaries that maps reviewers and asins to indexes
reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}
asinMap = {asin: i for i, asin in enumerate(asins)}

# Compute the pairwise cosine similarity between users
user_similarity = cosine_similarity(utilityMatrix, dense_output=False).astype("float32")

hoodies = metadata[metadata['category'].apply(lambda c: "Hoodie" in c)]
hoodieAsins = hoodies["asin"]
testAsins = hoodieAsins[5:9]
testIndices = testAsins.index.to_list()

# Create an anonymous user with four reviews
anonymous_user = pd.DataFrame({'overall': [5, 5, 5, 5]}, index=testIndices).T
cols = [utilityMatrix.getcol(col) for col in testIndices]
relevantItems = hstack(cols)

# Compute the similarity between the anonymous user and every other user
user_similarity_scores = cosine_similarity(relevantItems, anonymous_user.fillna(0), dense_output=False)
print(user_similarity_scores.shape)
# Get the k most similar users to the anonymous user

similar_user_indices = np.sort(-user_similarity_scores)
print(similar_user_indices, similar_user_indices.shape)

(141755, 1)
[[-0.]
 [-0.]
 [-0.]
 ...
 [-0.]
 [-0.]
 [-0.]] (141755, 1)
